In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Reader, Dataset,SlopeOne, SVD ,accuracy,Trainset,KNNWithMeans,KNNBasic,BaselineOnly,CoClustering,SVDpp,NMF,KNNBaseline
from surprise.model_selection import train_test_split,cross_validate,GridSearchCV
import random
from sklearn.model_selection import train_test_split as sklearn_train_test_split

In [5]:
### Merged Data Shape (100480507, 3)
df1 = pd.read_csv('combined_data_1.txt', names = ['Cust_Id', 'Rating'], usecols = [0,1])
df1.shape

(24058263, 2)

In [ ]:
df2 = pd.read_csv('combined_data_2.txt', names = ['Cust_Id', 'Rating'], usecols = [0,1])
df3 = pd.read_csv('combined_data_3.txt', names = ['Cust_Id', 'Rating'], usecols = [0,1])
df4 = pd.read_csv('combined_data_4.txt', names = ['Cust_Id', 'Rating'], usecols = [0,1])
df = pd.concat([df1, df2, df3, df4], axis = 0)
df =  pd.read_csv('Netflix_Data/Merged_final_data.csv',index_col=None)

In [2]:
### Merged Data Shape (100480507, 3)
df =  pd.read_csv('Netflix_Data/Merged_final_data.csv',index_col=None)
df.shape

(100480507, 3)

In [3]:
df.head()

,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1


### To run the GridSearch on different methods, I am going to choose a small chunk of data and do my gridsearch and later I will use those parameters to fit on bigger data size

In [4]:
# Data) Group_1: Smallest group to fast implementation of different algorithms

Data_m500_C10000 = df[df['Movie_Id']<=500]

In [5]:
#Data_m500_C10000.to_csv('Data_m500_C10000.csv',index=False)

In [5]:
# Sorted Customers Ids
Cust_set = sorted(set(df['Cust_Id'].unique()))

In [6]:
# random 1000 users
rand_sel = random.sample(Cust_set,1000)

In [7]:
Data_m500_C10000 =Data_m500_C10000[Data_m500_C10000['Cust_Id'].isin(rand_sel)]
Data_m500_C10000.shape

(5937, 3)

In [8]:
# later I will use Gridseach to find optimal parameters, to avoid double dipping I put aside 10% of data az a test_set
train_data , test_data = sklearn_train_test_split(Data_m500_C10000,test_size=0.1)

In [9]:
### preparing data to work in the Surprise package
reader = Reader()

data = Dataset.load_from_df(train_data[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [14]:
knn_test_data =  Dataset.load_from_df(test_data[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [10]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [11]:
train_set.n_users , train_set.n_items , train_set.n_ratings

(707, 313, 4274)

In [17]:
sim_options = sim_options = {'name': 'pearson_baseline',
               'shrinkage': 10,'user_based':False}

In [18]:
knn = KNNWithMeans(k=10,sim_options=sim_options,min_k=3)

In [19]:
model = knn.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [25]:
prediction = model.test(val_set)

In [21]:
accuracy.rmse(prediction)

RMSE: 1.0707


1.0706822289431968

In [22]:
# lets do a GridSearch to findout which parameters value works better
param_grid = {'k':[5,10,15,20],'min_k':[1,2,3,5],'sim_options': {'name':['pearson_baseline','MSD'],'user_based':[True,False],'shrinkage':[10,50,100]}} 

In [23]:
GS = GridSearchCV(KNNWithMeans,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True)

In [24]:
GS.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [26]:
GS.best_params['rmse']

{'k': 20,
 'min_k': 5,
 'sim_options': {'name': 'MSD', 'user_based': False, 'shrinkage': 10}}

In [27]:
GS.best_score

{'rmse': 1.0444106948345984, 'mae': 0.8238950155719185}

In [33]:
# construct a surprise test_set
trainss_set , test_set = train_test_split(knn_test_data,test_size=0.9999)

In [36]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [37]:
accuracy.rmse(prediction)

RMSE: 0.9580


0.9579952819646282

In [46]:
### KNN_BASELINE
knn_baseline =KNNBaseline(k=10,sim_options=sim_options,min_k=3)
param_grid = {'k':[15,20,25,30],'min_k':[2,5,10],'sim_options': {'name':['pearson_baseline','MSD','cosine'],'user_based':[True,False],'shrinkage':[10,50,100]}} 
GS = GridSearchCV(KNNBaseline,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [54]:
knn_baseline.fit(train_set)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [55]:
prediction = knn_baseline.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0216


1.0216161198118496

In [47]:
GS.fit(data)    

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [50]:
GS.best_params     ### KNN BASELINE

{'rmse': {'k': 25,
  'min_k': 10,
  'sim_options': {'name': 'pearson_baseline',
   'user_based': False,
   'shrinkage': 10}}}

In [51]:
GS.best_score

{'rmse': 1.0091090369908768}

In [52]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [53]:
accuracy.rmse(prediction)

RMSE: 0.9081


0.9081282842427049

In [12]:
###### SVD 
svd = SVD()
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0226


1.02260866807732

In [19]:
train_set.n_users,train_set.n_items,train_set.n_ratings

(707, 313, 4274)

In [13]:
my_test = train_set.build_anti_testset()

In [22]:
pred = model.test(my_test)

In [60]:
param_grid ={'n_factors':[20,50,100],'lr_all':[0.005,0.01,0.001],'reg_all':[0.02,0.01,0]} 
GS = GridSearchCV(SVD,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [61]:
GS.fit(data)

In [62]:
GS.best_params  ##SVD

{'rmse': {'n_factors': 20, 'lr_all': 0.005, 'reg_all': 0.01}}

In [63]:
GS.best_score

{'rmse': 1.0025465044748745}

In [64]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [65]:
accuracy.rmse(prediction)

RMSE: 0.8945


0.8945494490053538

In [ ]:
### BASELINE ONLY

In [66]:
basic = BaselineOnly()
model = basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
RMSE: 1.0006


1.0005712842595251

In [ ]:
#### SVD++

In [67]:
svdpp = SVDpp()
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9954


0.995370503053884

In [68]:
param_grid ={'n_factors':[20,50,100],'lr_all':[0.005,0.01,0.001],'reg_all':[0.02,0.01,0]} 
GS = GridSearchCV(SVD,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [69]:
GS.fit(data)

In [70]:
GS.best_params     ### SVD++

{'rmse': {'n_factors': 20, 'lr_all': 0.005, 'reg_all': 0}}

In [71]:
GS.best_score

{'rmse': 1.0039568808819568}

In [72]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [73]:
accuracy.rmse(prediction)

RMSE: 0.8906


0.8905708152170263

In [74]:
### Slope-One

In [76]:
slope = SlopeOne()
model = slope.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0675


1.0674752415859448

In [77]:
### NMF

In [78]:
nmf = NMF()
model = nmf.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.1603


1.1602621077490962

In [79]:
### Co-clustering

In [80]:
co = CoClustering()
model = co.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0806


1.0805885394951966

In [88]:
param_grid ={'n_cltr_u':[2,3,5,10],'n_cltr_i':[2,3,5,10,20]} 
GS = GridSearchCV(CoClustering,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [89]:
GS.fit(data)

In [90]:
GS.best_params    # Coclustering

{'rmse': {'n_cltr_u': 2, 'n_cltr_i': 2}}

In [91]:
GS.best_score

{'rmse': 1.0898561137050264}

In [92]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [93]:
accuracy.rmse(prediction)

RMSE: 0.9959


0.9959275258000879

### Now lets work with 700 movies and all users that we have

In [251]:
Data_m700= df[df['Movie_Id']<=700]

In [252]:
Data_m700.shape

(3626024, 3)

In [107]:
rand_sel = random.sample(Cust_set,5000)
Data_m700_C5000 =Data_m700_C5000[Data_m700_C5000['Cust_Id'].isin(rand_sel)]
Data_m700_C5000.shape

(37370, 3)

In [253]:
reader = Reader()

data = Dataset.load_from_df(Data_m700[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [114]:
#knn_test_data =  Dataset.load_from_df(test_data[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [254]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [255]:
train_set.n_users , train_set.n_items , train_set.n_ratings

(369993, 700, 2900819)

In [121]:
# sim_options = sim_options = {'name': 'pearson_baseline',
#                'shrinkage': 10,'user_based':False}

In [122]:
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=2)

In [123]:
# model = knn.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [124]:
# prediction = model.test(val_set)

In [125]:
accuracy.rmse(prediction)

RMSE: 1.0321


1.0321464168326322

In [127]:
param_grid = {'k':[10,15,20,25],'min_k':[3,5,8],'sim_options': {'name':['pearson_baseline','MSD'],'user_based':[True,False],'shrinkage':[10,50]}} 

In [130]:
GS = GridSearchCV(KNNWithMeans,param_grid=param_grid,cv=5,n_jobs=-1,measures=['rmse'],refit=True,return_train_measures=True)

In [131]:
GS.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [132]:
GS.best_params['rmse']

{'k': 25,
 'min_k': 5,
 'sim_options': {'name': 'MSD', 'user_based': False, 'shrinkage': 10}}

In [133]:
GS.best_score

{'rmse': 0.9861761303385833}

In [134]:
# construct a surprise test_set
trainss_set , test_set = train_test_split(knn_test_data,test_size=0.9999)

In [135]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [136]:
accuracy.rmse(prediction)

RMSE: 0.9601


0.9600725953869318

In [137]:
### KNN_BASELINE
knn_baseline =KNNBaseline(k=10,sim_options=sim_options,min_k=3)
param_grid = {'k':[20,25,30],'min_k':[3,5,7],'sim_options': {'name':['pearson_baseline','MSD'],'user_based':[True,False],'shrinkage':[10,50]}} 
GS = GridSearchCV(KNNBaseline,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [138]:
knn_baseline.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [139]:
prediction = knn_baseline.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0165


1.0164968115430255

In [140]:
GS.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [141]:
GS.best_params

{'rmse': {'k': 30,
  'min_k': 7,
  'sim_options': {'name': 'pearson_baseline',
   'user_based': True,
   'shrinkage': 50}}}

In [142]:
GS.best_score

{'rmse': 0.9686033325888026}

In [144]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [145]:
accuracy.rmse(prediction)

RMSE: 0.9402


0.9401552561666605

In [146]:
###### SVD 
svd = SVD()
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9908


0.9908079966935951

In [147]:
param_grid ={'n_factors':[20,50,100],'lr_all':[0.005,0.01,0.001],'reg_all':[0.02,0.01,0]} 
GS = GridSearchCV(SVD,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [148]:
GS.fit(data)

In [149]:
GS.best_params

{'rmse': {'n_factors': 20, 'lr_all': 0.005, 'reg_all': 0.02}}

In [150]:
GS.best_score

{'rmse': 0.9688693964774316}

In [151]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [152]:
accuracy.rmse(prediction)

RMSE: 0.9519


0.9519079102042844

In [153]:
### BASELINE ONLY

In [154]:
basic = BaselineOnly()
model = basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
RMSE: 0.9813


0.9812976427083236

In [155]:
### SVD++

In [156]:
svdpp = SVDpp()
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9730


0.9730136115362367

In [157]:
param_grid ={'n_factors':[15,20,50,],'lr_all':[0.005,0.01,0.001],'reg_all':[0.02,0.03,0]} 
GS = GridSearchCV(SVD,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [158]:
GS.fit(data)

In [159]:
GS.best_params

{'rmse': {'n_factors': 20, 'lr_all': 0.005, 'reg_all': 0.03}}

In [160]:
GS.best_score

{'rmse': 0.9675263191283673}

In [161]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [162]:
accuracy.rmse(prediction)

RMSE: 0.9530


0.9530044587103452

In [166]:
### SLOPE-ONE

In [167]:
slope = SlopeOne()
model = slope.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0383


1.0382939380945042

In [168]:
### NMF

In [169]:
### NMF
nmf = NMF()
model = nmf.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0975


1.0974541539681797

In [170]:
### Co-clustering

In [171]:
### Co-clustering
co = CoClustering()
model = co.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0591


1.0591412605477484

In [173]:
param_grid ={'n_cltr_u':[2,3,5],'n_cltr_i':[2,3,5,10]} 
GS = GridSearchCV(CoClustering,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [174]:
GS.fit(data)

In [175]:
GS.best_score

{'rmse': 1.0364553333612025}

In [176]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [177]:
accuracy.rmse(prediction)

RMSE: 1.0017


1.0017289948836394

### let's make our Data bigger, this time I choose 1000 movies and 10000 customers. This time we just implement KNN and SVD

In [179]:
Data_m1000_C10000 = df[df['Movie_Id']<=1000]

In [180]:
Data_m1000_C10000.shape

(5010199, 3)

In [181]:
rand_sel = random.sample(Cust_set,10000)

In [182]:
Data_m1000_C10000 =Data_m1000_C10000[Data_m1000_C10000['Cust_Id'].isin(rand_sel)]
Data_m1000_C10000.shape

(102998, 3)

In [188]:
# later I will use Gridseach to find optimal parameters, to avoid double dipping I put aside 10% of data az a test_set
train_data , test_data = sklearn_train_test_split(Data_m1000_C10000,test_size=0.05)

In [189]:
reader = Reader()

data = Dataset.load_from_df(train_data[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [190]:
knn_test_data =  Dataset.load_from_df(test_data[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [191]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [192]:
train_set.n_users , train_set.n_items , train_set.n_ratings

(8050, 964, 78278)

In [193]:
sim_options = sim_options = {'name': 'pearson_baseline',
               'shrinkage': 10,'user_based':False}

In [194]:
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=5)

In [195]:
model = knn.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [196]:
prediction = model.test(val_set)

In [197]:
accuracy.rmse(prediction)

RMSE: 0.9689


0.9689320883732929

In [198]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [199]:
param_grid = {'k':[20,25,30],'min_k':[3,5,8],'sim_options': {'name':['pearson_baseline','MSD'],'user_based':[True,False],'shrinkage':[10,50]}} 

In [200]:
GS = GridSearchCV(KNNWithMeans,param_grid=param_grid,cv=5,n_jobs=-1,measures=['rmse'],refit=True,return_train_measures=True)

In [201]:
GS.fit(data)

C:\Users\hesam\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Computing the msd similarity matrix...
Done computing similarity matrix.


In [202]:
GS.best_params['rmse']

{'k': 20,
 'min_k': 8,
 'sim_options': {'name': 'MSD', 'user_based': False, 'shrinkage': 10}}

In [203]:
GS.best_score

{'rmse': 0.9714902698257889}

In [204]:
# construct a surprise test_set
trainss_set , test_set = train_test_split(knn_test_data,test_size=0.9999)

In [205]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [206]:
accuracy.rmse(prediction)

RMSE: 0.9660


0.9660055270571365

In [207]:
### KNN_BASELINE
knn_baseline =KNNBaseline(k=10,sim_options=sim_options,min_k=3)
param_grid = {'k':[20,25],'min_k':[5,7],'sim_options': {'name':['pearson_baseline','MSD'],'user_based':[True,False],'shrinkage':[10,50]}} 
GS = GridSearchCV(KNNBaseline,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [208]:
knn_baseline.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [210]:
prediction = knn_baseline.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9729


0.9728924124606574

In [211]:
GS.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [212]:
GS.best_params['rmse']

{'k': 25,
 'min_k': 7,
 'sim_options': {'name': 'pearson_baseline',
  'user_based': True,
  'shrinkage': 50}}

In [213]:
GS.best_score

{'rmse': 0.9564174140220464}

In [214]:
prediction = knn_baseline.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9729


0.9728924124606574

In [215]:
###### SVD 
svd = SVD()
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9671


0.9670714616934553

In [216]:
param_grid ={'n_factors':[15,20,50],'lr_all':[0.005,0.01,0.001],'reg_all':[0.02,0.01]} 
GS = GridSearchCV(SVD,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [217]:
GS.fit(data)

In [218]:
GS.best_params

{'rmse': {'n_factors': 15, 'lr_all': 0.005, 'reg_all': 0.02}}

In [219]:
GS.best_score

{'rmse': 0.9629169672247718}

In [220]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [221]:
accuracy.rmse(prediction)

RMSE: 0.9600


0.9600124039449857

In [222]:
### SVD++

In [223]:
svdpp = SVDpp()
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9463


0.9462873904768228

In [224]:
param_grid ={'n_factors':[10,15,20],'lr_all':[0.005,0.01,0.001],'reg_all':[0.02,0.03]} 
GS = GridSearchCV(SVD,param_grid=param_grid,cv=5,n_jobs=-1,refit=True,return_train_measures=True,measures=['rmse'])

In [225]:
GS.fit(data)

In [226]:
GS.best_params

{'rmse': {'n_factors': 10, 'lr_all': 0.005, 'reg_all': 0.03}}

In [227]:
GS.best_score

{'rmse': 0.9607100974276047}

In [228]:
prediction = GS.best_estimator['rmse'].test(test_set)

In [229]:
accuracy.rmse(prediction)

RMSE: 0.9595


0.9595231061661629

### This Time I want to keep movie number same as 1000 but increase the number of custemers to 15000 

In [231]:
Data_m1000_C15000 = df[df['Movie_Id']<=1000]

In [232]:
Data_m1000_C15000.shape

(5010199, 3)

In [233]:
rand_sel = random.sample(Cust_set,15000)

In [234]:
Data_m1000_C15000 =Data_m1000_C15000[Data_m1000_C15000['Cust_Id'].isin(rand_sel)]
Data_m1000_C15000.shape

(157486, 3)

In [235]:
# later I will use Gridseach to find optimal parameters, to avoid double dipping I put aside 10% of data az a test_set
train_data , test_data = sklearn_train_test_split(Data_m1000_C15000,test_size=0.05)

In [236]:
reader = Reader()

data = Dataset.load_from_df(train_data[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [237]:
knn_test_data =  Dataset.load_from_df(test_data[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [238]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [239]:
train_set.n_users , train_set.n_items , train_set.n_ratings

(12085, 975, 119688)

In [240]:
sim_options = sim_options = {'name': 'pearson_baseline',
               'shrinkage': 10,'user_based':False}

In [241]:
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=5)

In [242]:
model = knn.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [243]:
prediction = model.test(val_set)

In [244]:
accuracy.rmse(prediction)

RMSE: 0.9752


0.9752405931420735

In [245]:
# construct a surprise test_set
trainss_set , test_set = train_test_split(knn_test_data,test_size=0.9999)

In [248]:
param_grid = {'k':[20,25,30],'min_k':[3,5,8],'sim_options': {'name':['pearson_baseline','MSD'],'user_based':[True,False],'shrinkage':[10,50]}} 

In [249]:
GS = GridSearchCV(KNNWithMeans,param_grid=param_grid,cv=5,n_jobs=-1,measures=['rmse'],refit=True,return_train_measures=True)

In [250]:
GS.fit(data)     #### I got an error

KeyboardInterrupt: 

### I will use the parameters I found to train bigger Data size 
### Now I want to use the best parameters I found on 700 movies to train the model on all customers

In [3]:
Data_m700= df[df['Movie_Id']<=700]

In [4]:
Data_m700.shape

(3626024, 3)

In [5]:
reader = Reader()

data = Dataset.load_from_df(Data_m700[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [6]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [7]:
### KNN BASIC
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_basic =KNNBasic(k=20,sim_options=sim_options,min_k=7)
model = knn_basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0186


1.0186002908119023

In [8]:
### KNN BASELINE
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_baseline =KNNBaseline(k=20,sim_options=sim_options,min_k=7)
model = knn_baseline.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9589


0.9588904784718942

In [265]:
###### SVD 
svd = SVD(n_factors=20,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9704


0.9703801360443698

In [266]:
#### BASELINE ONLY
basic = BaselineOnly()
model = basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
RMSE: 0.9707


0.9706636715499066

In [267]:
### SVD++
svdpp = SVDpp(n_factors=20,lr_all=0.005,reg_all=0.03)
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9557


0.9556559126803662

In [270]:
### SLOPE-ONE
slope = SlopeOne()
model = slope.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9966


0.9966212378894829

In [271]:
### NMF
nmf = NMF()
model = nmf.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0545


1.0545040701485553

In [272]:
### Co-clustering
co = CoClustering(n_cltr_i=2,n_cltr_u=2)
model = co.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0246


1.0245902010937715

In [273]:
### to be able to run knn.algorithms I should limit the number of users: users = 20000

In [274]:
rand_sel = random.sample(Cust_set,20000)
Data_m700_C20000 = Data_m700[Data_m700['Cust_Id'].isin(rand_sel)]
Data_m700_C20000.shape

(149303, 3)

In [275]:
reader = Reader()

data = Dataset.load_from_df(Data_m700_C20000[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [276]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [277]:
train_set.n_users , train_set.n_items , train_set.n_ratings

(15377, 696, 119442)

In [278]:
sim_options = sim_options = {'name': 'pearson_baseline','shrinkage': 50,'user_based':True}

In [279]:
knn_baseline = KNNBaseline(k=25,min_k=7,sim_options=sim_options)

In [280]:
model = knn_baseline.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [281]:
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9608


0.9607831489426137

In [282]:
## KNN WITH MEANS
sim_options = sim_options = {'name': 'MSD','user_based':False}
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=8)
model = knn.fit(train_set)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [283]:
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9762


0.9761784505016443

In [ ]:
### lets increase use size to 30000

In [284]:
rand_sel = random.sample(Cust_set,30000)
Data_m700_C30000 = Data_m700[Data_m700['Cust_Id'].isin(rand_sel)]
Data_m700_C30000.shape

(223875, 3)

In [285]:
reader = Reader()

data = Dataset.load_from_df(Data_m700_C30000[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [286]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [287]:
train_set.n_users , train_set.n_items , train_set.n_ratings

(23139, 700, 179100)

In [288]:
sim_options = sim_options = {'name': 'pearson_baseline','shrinkage': 50,'user_based':True}

In [289]:
knn_baseline = KNNBaseline(k=25,min_k=7,sim_options=sim_options)

In [290]:
model = knn_baseline.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [291]:
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9602


0.9602041987708557

In [292]:
## KNN WITH MEANS
sim_options = sim_options = {'name': 'MSD','user_based':False}
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=8)
model = knn.fit(train_set)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [293]:
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9773


0.9772877867316165

In [ ]:
### lets increase use size to 50000

In [294]:
rand_sel = random.sample(Cust_set,50000)
Data_m700_C50000 = Data_m700[Data_m700['Cust_Id'].isin(rand_sel)]
Data_m700_C50000.shape

(372941, 3)

In [295]:
reader = Reader()

data = Dataset.load_from_df(Data_m700_C50000[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [296]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [297]:
train_set.n_users , train_set.n_items , train_set.n_ratings

(38442, 700, 298352)

In [304]:
sim_options = sim_options = {'name': 'MSD','user_based':False}
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=8)
model = knn.fit(train_set)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [305]:
reader = Reader()

data = Dataset.load_from_df(Data_m700[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [306]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [307]:
train_set.n_users , train_set.n_items , train_set.n_ratings

(370033, 700, 2900819)

In [308]:
sim_options = sim_options = {'name': 'MSD','user_based':False}
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=8)
model = knn.fit(train_set)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [309]:
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9708


0.9707863684951246

### We decide to train and test our model on 3 different data-set. this time I choose different 700 movies.
### group 2 movies 1000-1700

In [9]:
Data2_m700 = df[df['Movie_Id']>1000]
Data2_m700 = Data2_m700[Data2_m700['Movie_Id']<=1700]
Data2_m700.shape

(3492051, 3)

In [10]:
reader = Reader()

data = Dataset.load_from_df(Data2_m700[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [11]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [322]:
###### SVD 
svd = SVD(n_factors=20,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9437


0.9436852421716381

In [12]:
### KNN BASIC
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_basic =KNNBasic(k=20,sim_options=sim_options,min_k=7)
model = knn_basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0082


1.008171158706824

In [13]:
### KNN BASELINE
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_baseline =KNNBaseline(k=20,sim_options=sim_options,min_k=7)
model = knn_baseline.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9317


0.9316885475257562

In [323]:
#### BASELINE ONLY
basic = BaselineOnly()
model = basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
RMSE: 0.9440


0.9440263459931508

In [324]:
### SVD++
svdpp = SVDpp(n_factors=20,lr_all=0.005,reg_all=0.03)
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9224


0.9223633616975427

In [325]:
### SLOPE-ONE
slope = SlopeOne()
model = slope.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9685


0.9685144084357385

In [326]:
### NMF
nmf = NMF()
model = nmf.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0311


1.031123318938917

In [327]:
### Co-clustering
co = CoClustering(n_cltr_i=2,n_cltr_u=2)
model = co.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9913


0.9913413103387431

In [328]:
### KNN WITHMEANS
sim_options = sim_options = {'name': 'MSD','user_based':False}
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=8)
model = knn.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9474


0.947352912710028

### group 3 movies 2000-2700

In [14]:
Data3_m700 = df[df['Movie_Id']>2000]
Data3_m700 = Data3_m700[Data3_m700['Movie_Id']<=2700]
Data3_m700.shape

(3756961, 3)

In [15]:
reader = Reader()

data = Dataset.load_from_df(Data3_m700[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [16]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [17]:
train_set.n_items,train_set.n_ratings,train_set.n_users

(700, 3005568, 386466)

In [18]:
### KNN BASIC
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_basic =KNNBasic(k=20,sim_options=sim_options,min_k=7)
model = knn_basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0234


1.023366107656062

In [19]:
### KNN BASELINE
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_baseline =KNNBaseline(k=20,sim_options=sim_options,min_k=7)
model = knn_baseline.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9387


0.9387307612283162

In [334]:
###### SVD 
svd = SVD(n_factors=20,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9479


0.9479233063318543

In [335]:
#### BASELINE ONLY
baseline = BaselineOnly()
model = basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
RMSE: 0.9503


0.9503286615027874

In [336]:
### SVD++
svdpp = SVDpp(n_factors=20,lr_all=0.005,reg_all=0.03)
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9282


0.9282349138416324

In [337]:
### SLOPE-ONE
slope = SlopeOne()
model = slope.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9753


0.9752819974279513

In [338]:
### NMF
nmf = NMF()
model = nmf.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0335


1.033487153213744

In [339]:
### Co-clustering
co = CoClustering(n_cltr_i=2,n_cltr_u=2)
model = co.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0078


1.0078353333439491

In [342]:
### KNN WITHMEANS
sim_options = sim_options = {'name': 'MSD','user_based':False}
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=8)
model = knn.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9517


0.9516756105040494

### 1000 movies

In [5]:
Data_m1000 = df[df['Movie_Id']<=1000]

In [6]:
Data_m1000.shape

(5010199, 3)

In [40]:
reader = Reader()

data = Dataset.load_from_df(Data_m1000[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [41]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [42]:
train_set.n_items,train_set.n_ratings,train_set.n_users

(1000, 4008159, 390582)

In [11]:
###### SVD 
svd = SVD(n_factors=20,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9513


0.9512650718379673

In [12]:
#### BASELINE ONLY
basic = BaselineOnly()
model = basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
RMSE: 0.9559


0.9558636424950209

In [13]:
### SVD++
svdpp = SVDpp(n_factors=20,lr_all=0.005,reg_all=0.03)
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9408


0.9407554254459533

In [14]:
### SLOPE-ONE
slope = SlopeOne()
model = slope.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9734


0.9733633299638529

In [15]:
### NMF
nmf = NMF()
model = nmf.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0247


1.024704944620662

In [16]:
### Co-clustering
co = CoClustering(n_cltr_i=2,n_cltr_u=2)
model = co.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9993


0.9992830651900744

In [44]:
### KNN WITHMEANS
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn = KNNWithMeans(k=25,sim_options=sim_options,min_k=7)
model = knn.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9519


0.9519450264117444

In [45]:
### KNN BASIC
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_basic =KNNBasic(k=25,sim_options=sim_options,min_k=7)
model = knn_basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9955


0.9955377348133618

In [43]:
### KNN BASELINE
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_baseline =KNNBaseline(k=25,sim_options=sim_options,min_k=7)
model = knn_baseline.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9388


0.9388321631168203

### Group_2 


In [48]:
Data2_m1000 = df[df['Movie_Id']>1000]
Data2_m1000 = Data2_m1000[Data2_m1000['Movie_Id']<=2000]
Data2_m1000.shape

(5307070, 3)

In [49]:
reader = Reader()

data = Dataset.load_from_df(Data2_m1000[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [50]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [51]:
train_set.n_items,train_set.n_ratings,train_set.n_users

(1000, 4245656, 408691)

In [52]:
###### SVD 
svd = SVD(n_factors=20,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9358


0.9358030828848581

In [53]:
#### BASELINE ONLY
basic = BaselineOnly()
model = basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
RMSE: 0.9436


0.9436484267958758

In [54]:
### SVD++
svdpp = SVDpp(n_factors=20,lr_all=0.005,reg_all=0.03)
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9207


0.9207232864294953

In [55]:
### SLOPE-ONE
slope = SlopeOne()
model = slope.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9576


0.9575776501580092

In [56]:
### NMF
nmf = NMF()
model = nmf.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0010


1.0009976932185207

In [57]:
### Co-clustering
co = CoClustering(n_cltr_i=2,n_cltr_u=2)
model = co.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9855


0.9855350262430876

In [61]:
### KNN WITHMEANS
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn = KNNWithMeans(k=25,sim_options=sim_options,min_k=8)
model = knn.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9407


0.9406616420258341

In [62]:
### KNN BASIC
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_basic =KNNBasic(k=25,sim_options=sim_options,min_k=7)
model = knn_basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9930


0.9929900713223555

In [63]:
### KNNBASELINE
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_baseline =KNNBaseline(k=25,sim_options=sim_options,min_k=7)
model = knn_baseline.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9231


0.9231077218570384

In [4]:
#group_3 m1000
Data3_m1000 = df[df['Movie_Id']>2000]
Data3_m1000 = Data3_m1000[Data3_m1000['Movie_Id']<=3000]
Data3_m1000.shape

(5227456, 3)

In [5]:
reader = Reader()

data = Dataset.load_from_df(Data3_m1000[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [6]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [7]:
train_set.n_items,train_set.n_ratings,train_set.n_users

(1000, 4181964, 406826)

In [8]:
###### SVD 
svd = SVD(n_factors=20,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9417


0.9416979703827946

In [69]:
#### BASELINE ONLY
basic = BaselineOnly()
model = basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
RMSE: 0.9442


0.944245896538629

In [70]:
### SVD++
svdpp = SVDpp(n_factors=20,lr_all=0.005,reg_all=0.03)
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9246


0.9245930649521887

In [71]:
### SLOPE-ONE
slope = SlopeOne()
model = slope.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9615


0.9615481016729471

In [72]:
### NMF
nmf = NMF()
model = nmf.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 1.0068


1.0067516512791579

In [73]:
### Co-clustering
co = CoClustering(n_cltr_i=2,n_cltr_u=2)
model = co.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9910


0.9910433892189945

In [74]:
### KNN WITHMEANS
sim_options = sim_options = {'name': 'MSD','user_based':False}
knn = KNNWithMeans(k=20,sim_options=sim_options,min_k=8)
model = knn.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9408


0.9408480078925173

In [75]:
### KNN BASIC
sim_options = sim_options = {'name': 'cosine','user_based':False}
knn_basic =KNNBasic(k=10,sim_options=sim_options,min_k=3)
model = knn_basic.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0405


1.0404684231834613

In [76]:
### KNNBASELINE
sim_options = sim_options = {'name': 'pearson_baseline','user_based':False}
knn_baseline =KNNBaseline(k=25,sim_options=sim_options,min_k=7)
model = knn_baseline.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9266


0.926627882051687

### train-test for SVM on bigger Data-set 
### SVD for 3000 movies

In [351]:
Data_m3000 = df[df['Movie_Id']<3000]

In [352]:
Data_m3000.shape

(15543024, 3)

In [353]:
reader = Reader()

data = Dataset.load_from_df(Data_m3000[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [354]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [355]:
train_set.n_items,train_set.n_ratings,train_set.n_users

(2999, 12434419, 455788)

In [356]:
###### SVD 
svd = SVD(n_factors=20,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9095


0.9094673968743336

In [357]:
svdpp = SVDpp(n_factors=20,lr_all=0.005,reg_all=0.03)
model = svdpp.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.9198


0.9198311273720563

### SVD for 5000 movies

In [359]:
Data_m5000 = df[df['Movie_Id']<5000]

In [363]:
Data_m5000.shape

(27009333, 3)

In [364]:
reader = Reader()

data = Dataset.load_from_df(Data_m5000[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [365]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [366]:
train_set.n_items,train_set.n_ratings,train_set.n_users

(4999, 21607466, 469284)

In [367]:
###### SVD 
svd = SVD(n_factors=20,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.8959


0.895899679290832

In [368]:
svd = SVD(n_factors=30,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.8945


0.8944779884755923

#### SVD for 7000 movies

In [4]:
Data_m7000 = df[df['Movie_Id']<=7000]

In [5]:
Data_m7000.shape

(39687830, 3)

In [6]:
reader = Reader()

data = Dataset.load_from_df(Data_m7000[['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [7]:
train_set , val_set = train_test_split(data,test_size=0.2)

In [8]:
train_set.n_items,train_set.n_ratings,train_set.n_users

(7000, 31750264, 475300)

In [9]:
svd = SVD(n_factors=30,lr_all=0.005,reg_all=0.02)
model = svd.fit(train_set)
prediction = model.test(val_set)
accuracy.rmse(prediction)

RMSE: 0.8781


0.8780509540124299